In [1]:
# (a) Join the What’s Cooking competition on Kaggle. Download the training and test data (in
# .json). The competition page describes how these files are formatted.
import pandas as pd
import numpy as np
train_data = pd.read_json("train.json")
test_data = pd.read_json("test.json")

In [2]:
# (b) Tell us about the data. How many samples (dishes) are there in the training set? How many
# categories (types of cuisine)? Use a list to keep all the unique ingredients appearing in the
# training set. How many unique ingredients are there?
num_dish = train_data.shape[0]
print "number of dish is",num_dish
print "unique types of cusines is",train_data['cuisine'].describe()['unique']
train_ingredients = pd.DataFrame(train_data['ingredients'].tolist()).stack()
test_ingredients = pd.DataFrame(test_data['ingredients'].tolist()).stack()
print "number of unique ingredients in training set is",train_ingredients.describe()['unique']


number of dish is 39774
unique types of cusines is 20
number of unique ingredients in training set is 6714


In [3]:
# (c) Represent each dish by a binary ingredient feature vector. Suppose there are d different ingredients
# in total from the training set, represent each dish by a 1×d binary ingredient vector
# x, where xi = 1 if the dish contains ingredient i and xi = 0 otherwise. For example, suppose
# all the ingredients we have in the training set are { beef, chicken, egg, lettuce, tomato, rice }
# and the dish is made by ingredients { chicken, lettuce, tomato, rice }, then the dish could be
# represented by a 6×1 binary vector [0, 1, 0, 1, 1, 1] as its feature or attribute. Use n ×d feature
# matrix to represent all the dishes in training set and test set, where n is the number of dishes.

In [4]:
######Method 1#####  >1 hour
#train= train_data['ingredients'].str.join(sep='*').str.get_dummies(sep='*') --very slow
#test = test_data['ingredients'].str.join(sep='*').str.get_dummies(sep='*')

In [5]:
# # ######Method 2#####  ~15 min on 8GB i5 machine
# # train_ing= pd.get_dummies(train_ingredients)
# # test_ing= pd.get_dummies(test_ingredients)
# # train= [train_ing.loc[i].sum(axis=0).transpose() for i in range(num_dish)]
# # test= [test_ing.loc[i].sum(axis=0).transpose() for i in range(test_data.shape[0])]
# train_array = pd.DataFrame(train)
# test_array = pd.DataFrame(test)
# common_col = list(set(train_array.columns.values).intersection(set(test_array.columns.values)))
# train_df = train_array[common_col]
# test_df = test_array[common_col]

In [6]:
######Method 3#####  ~2 mins
train_ingredients_list = train_ingredients.unique().tolist()
test_ingredients_list = test_ingredients.unique().tolist()
common_list = list(set(train_ingredients_list).intersection(test_ingredients_list))
train_df = [np.isin(common_list,x) for x in train_data['ingredients']]
test_df = [np.isin(common_list,x) for x in test_data['ingredients']]

In [7]:
# (d) Using Naïve Bayes Classifier to perform 3 fold cross-validation on the training set and report
# your average classification accuracy. Try both Gaussian distribution prior assumption and
# Bernoulli distribution prior assumption.
# (e) For Gaussian prior and Bernoulli prior, which performs better in terms of cross-validation
# accuracy? Why? Please give specific arguments.
from sklearn.naive_bayes import GaussianNB,BernoulliNB
from sklearn import cross_validation
gaussian = GaussianNB()
bernoulli = BernoulliNB()
train_label = train_data['cuisine']
gaussian_scores = cross_validation.cross_val_score(gaussian,train_df,train_label,cv=3)
bernoulli_scores = cross_validation.cross_val_score(bernoulli,train_df,train_label,cv=3)
print "Gaussian average accuracy: ", np.mean(gaussian_scores)
print "Bernoulli average accuracy: ", np.mean(bernoulli_scores)

/Users/charles/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Gaussian average accuracy:  0.369814620175
Bernoulli average accuracy:  0.713983983649


In [8]:
# (f ) Using Logistic Regression Model to perform 3 fold cross-validation on the training set and
# report your average classification accuracy.
from sklearn import linear_model
logit = linear_model.LogisticRegression()

logit_scores = cross_validation.cross_val_score(logit,train_df,train_label,cv=3)
print "Logistic Regression average accuracy: ", np.mean(logit_scores)

Logistic Regression average accuracy:  0.773671913969


In [9]:
# (g) Train your best-performed classifier with all of the training data, and generate test labels on
# test set. Submit your results to Kaggle and report the accuracy
logit.fit(train_df,train_label)
results = logit.predict(test_df)

In [10]:
#score 0.78368 
out_df = pd.DataFrame(results,columns=['cuisine'])
out_df['id'] = test_data['id']
out_df.to_csv("prediction.csv",index = False)